# Reading CSV database

**1. Set `runtime type` to `GPU`.**

In [1]:
!git clone https://github.com/LeSnakk/TextReadabilityAnnotation

fatal: destination path 'TextReadabilityAnnotation' already exists and is not an empty directory.


In [4]:
import pandas as pd

In [2]:
database_path = '/content/TextReadabilityAnnotation/project-files/annotation-data-text/text-preprocessed/CLEAR_Corpus_6.01_shortened.csv'

In [5]:
data = pd.read_csv(database_path)

excerpts = data['Excerpt'].tolist()

excerpts = [excerpt.replace('\n', ' ') for excerpt in excerpts]

print(f'Loaded database with {len(excerpts)} excerpts.')

FileNotFoundError: [Errno 2] No such file or directory: '/content/TextReadabilityAnnotation/project-files/annotation-data-text/text-preprocessed/CLEAR_Corpus_6.01_shortened.csv'

# Setting up LLAMA model

In [5]:
!pip install transformers torch accelerate

In [6]:
!huggingface-cli login

print('Logged in as:')
!huggingface-cli whoami


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.ca

In [7]:
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf" # meta-llama/Llama-2-7b-hf
# model = "meta-llama/Llama-2-13b-chat-hf" # meta-llama/Llama-2-13b-hf

tokenizer = AutoTokenizer.from_pretrained(model, use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:671: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [8]:
from transformers import pipeline

llama_pipeline = pipeline(
    "text-generation",  # LLM task
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [9]:
def get_llama_response(prompt: str) -> None:

    sequences = llama_pipeline(
        prompt,
        do_sample=True,
        top_k=40,
        top_p=0.1,
        temperature=0.7,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=512,
    )
    generated_text = sequences[0]['generated_text']
    prompt_length = len(prompt)
    cleaned_text = generated_text[prompt_length:].strip()
    # print("Chatbot:", sequences[0]['generated_text'])
    return cleaned_text

In [10]:
prompt = 'To which model am I talking to?'
print(get_llama_response(prompt))

Answer: You are talking to the BERT model. BERT (Bidirectional Encoder Representations from Transformers) is a language model developed by Google in 2018. It is a pre-trained deep learning model that can be fine-tuned for a wide range of natural language processing (NLP) tasks, such as sentiment analysis, question-answering, and text classification. BERT is particularly useful for tasks that require a deep understanding of the nuances of language, such as language translation, summarization, and generation.


# Tasking the model

In [11]:
prompt = f'What is the following text about: "{excerpts[8]}"?'
print(get_llama_response(prompt))

A) The story of a young British lieutenant of cavalry during World War I
B) The description of a specific battle during World War I
C) The author's personal experiences during World War I
D) A fictional story set during World War I


In [12]:
for i in range(11):
  prompt = f'What is the following text about: "{excerpts[i]}"? Give only one precise answer not exceeding three sentences.'
  print(f'Text {i}:')
  print(get_llama_response(prompt))

Text 0:
The text is describing a winter-themed party where guests play a game in a transformed ballroom, with a snow-covered floor, snow-dusted plants, and crystal icicles. The game involves planting a flag at either the North Pole or the South Pole, which are designated areas at opposite ends of the room. The guests wear snowshoes to make the game more challenging.
Text 1:
The text is about Mrs. Fayre's reluctance to let her daughter Dolly go on a pleasure trip, but ultimately deciding to give in to her daughter's desire.
Text 2:
The text is about a tennis game between Patty and Roger, and their defeat against Elise and Sam Blaney. Patty blames Roger for their loss, saying he is too absorbed in his Love Game to play tennis with his old skill. Roger defends himself, saying he has been busy and is not yet fully recovered from being sick.
Text 3:
Mr. Grimes is going to Sir John Harthover's house.
The chimneys need to be swept.
Tom is offended by the groom's appearance.
Text 4:
The text d

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


The text is about the eve of the Battle of Verdun, a significant event in World War I. The automobile is carrying three people, including a young British lieutenant of cavalry who is driving. The text provides some details about the lieutenant's appearance, suggesting that he is a young man in his late teens.
Text 9:
The text is about a group of boys who stop at a telephone booth to make long distance calls to their families after leaving the capital.
Text 10:
The text describes a man's journey through an open wood and his discovery of a wide plain covered with blue grass and flowers. He finds a ring of footprints in the grass and flowers without any visible path leading to it. The man is intrigued and decides to hide and wait to discover the meaning of the circle.


In [ ]:
new_column = {'LLAMA Score': '', 'LLAMA Explanation': ''}

data = pd.read_csv(database_path)

data = data.append(new_column, ignore_index=True)

data.to_csv(database_path, index=False)
# 3226
for i in range(len(excerpts)):
  prompt = f'How would you rate the readability of the following text on a scale from 1 (easy) to 10 (hard): "{excerpts[i]}"?'
  print(f'Text {i}:')
  response = get_llama_response(prompt)
  print(response)
  numbers = ''.join(filter(str.isdigit, response))
  print(numbers)
  data.at[i, 'LLAMA Score'] = numbers
  data.at[i, 'LLAMA Explanation'] = response
  data.to_csv(database_path, index=False)

<ipython-input-30-c1b57e69301f>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(new_column, ignore_index=True)


Text 0:


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
I would rate the readability of this text as a 4 (medium) on the readability scale. The text contains some complex vocabulary and sentence structures, but it is still relatively easy to understand for a typical 8-10 year old reader. The story is engaging and relatable, and the language is simple and clear. However, there are some challenging words and phrases, such as "faithful," "take care of," and "adore," that may require some additional explanation or support for younger readers.
4810
Text 1701:


Text 1702:
I would rate the readability of this text as a 4 (medium) on the readability scale. The text contains some complex vocabulary and sentence structures, but it is still relatively easy to understand for a reader with a basic level of education. The story is straightforward and the language is simple, making it accessible to a wide range of readers. However, some of the sentences are longer and more complex than ot